# 02 - Notable Drivers vs Teammate


## Motivation

As explored in the previous notebook `01: Drivers vs. Teammate`, in Formula 1, one of the best indicators of a driver’s performance is how he fares against his teammate. After having compared **WinRatios** across all drivers over their entire careers and related them to **Number of Races** (total race entries), we will dive deeper into the topic.

The previous analysis masked an important detail: a driver's **WinRatio** can vary dramatically over time, depending on the season and the teammate he is paired with. By focusing on on a few notable drivers, we will see how their **Win Ratios** evolve year‑by‑year and with different teammates and reveal fine‑grained performance nuances.


## Method

The calculation of a driver’s **Win Ratio** follows the same procedure described in the previous notebook; the only change is that we now compute it **per driver per year**. We will then choose a driver of interest, identify their respective teammate for each season, and plot the yearly **WinRatio**s side‑by‑side to highlight how the driver’s performance varies across seasons and teammate pairings.


## Gathering data

Thanks to a database wrapper and a prepared SQL query, we can easily obtain an up‑to‑date table of every race result in F1 history. The query returns one row for each driver‑teammate pairing in every Grand Prix they entered. Each row includes information on:

- race
- team
- driver
- teammate
- result (`win`, `loss` or `draw`) indicating whether the driver finished _higher_, _lower_, or _tied_ with his teammate.
- (if a teammate DNFs, the race is recorded as a `win` for the driver; if both DNF, it’s recorded as a `draw`)

**Example**

If, in a particular race, **Verstappen** finishes first and his teammate **Pérez** finishes second, the result set will contain two rows:

| RaceID | Team     | Driver     | Teammate   | Result |
| ------ | -------- | ---------- | ---------- | ------ |
| 1036   | Red Bull | Verstappen | Pérez      | win    |
| 1036   | Red Bull | Pérez      | Verstappen | loss   |


In [ ]:
import pandas as pd

from f1_analysis.f1db import F1DB

f1db = F1DB()

df = pd.DataFrame(f1db.execute_sql_query("driver_vs_teammate_h2h.sql"))
df.columns = [
    "RaceId",
    "Year",
    "ConstructorId",
    "ConstructorName",
    "DriverId",
    "DriverAbbreviation",
    "DriverName",
    "TeammateId",
    "TeammateAbbreviation",
    "TeammateName",
    "Result",
]

## Processing the data

To compute each driver’s **WinRatio**, we first group the data by **DriverId** and **Year**. For each driver and year pairing, we calculate:

| Metric               | Definition                                                           |
| -------------------- | -------------------------------------------------------------------- |
| **NumberOfWins**     | Count of rows where `Result = 'win'`                                 |
| **NumberOfMatchups** | Total number of rows for the driver (each teammate‑race combination) |
| **NumberOfRaces**    | Count of distinct races the driver actually participated in          |

The distinction between **NumberOfMatchups** and **NumberOfRaces** matters because, in some eras of Formula 1, a team fielded more than two drivers. Counting every row would inflate the race total, since a single race can generate multiple matchup rows for the same driver.

### Example

In race 1, driver **Fagioli** faces three teammates — **Farina**, **Parnell**, and **Fangio**. The result set contains three rows for Fagioli:

| RaceID | Team       | Driver  | Teammate | Result |
| ------ | ---------- | ------- | -------- | ------ |
| 1      | Alfa Romeo | Fagioli | Farina   | loss   |
| 1      | Alfa Romeo | Fagioli | Parnell  | win    |
| 1      | Alfa Romeo | Fagioli | Fangio   | win    |

- **NumberOfMatchups** = 3 (one row per teammate)
- **NumberOfWins** = 2
- **WinRatio** = 2 / 3 ≈ 0.66
- **NumberOfRaces** = 1

When counting **NumberOfRaces**, all three rows correspond to a single race, so they contribute **1** to the race total. This ensures the win ratio reflects performance per matchup, while the race count reflects actual participation.


In [ ]:
win_ratios = (
    df.groupby(["DriverId", "Year"])
    .agg(
        DriverName=("DriverName", "first"),
        DriverAbbreviation=("DriverAbbreviation", "first"),
        NumberOfMatchups=("DriverId", "count"),
        NumberOfWins=("Result", lambda r: (r == "win").sum()),
        NumberOfRaces=("RaceId", pd.Series.nunique),
    )
    .reset_index()
)

win_ratios["WinRatio"] = (
    win_ratios["NumberOfWins"] / win_ratios["NumberOfMatchups"]
).round(4)

## Filtering the Data

After calculating the metrics, we need to isolate the records for a single driver and his teammates. We’ll encapsulate the logic in reusable functions so it can be applied to any driver of interest. First, we filter the dataset for the selected driver and keep his yearly **WinRatio**s. Next, for each season we identify all drivers who were paired with him and select their respective results; if a driver had multiple teammates in a season, we retain only the teammate with the highest **NumberOfRaces**.


In [ ]:
def get_driver_win_ratios(driver_id):
    return win_ratios[win_ratios["DriverId"] == driver_id]


def get_teammates_win_ratios(driver_id):
    teammates_by_year = df[df["DriverId"] == driver_id][
        ["TeammateId", "Year"]
    ].drop_duplicates()

    teammates_win_ratios = pd.merge(
        left=teammates_by_year,
        right=win_ratios,
        how="left",
        left_on=["TeammateId", "Year"],
        right_on=["DriverId", "Year"],
    ).drop("TeammateId", axis=1)

    return teammates_win_ratios.loc[
        teammates_win_ratios.groupby("Year")["NumberOfRaces"].idxmax()
    ].reset_index(drop=True)

## Plotting the data

Again, as a reusable function, we plot the driver and the teamates as two traces on a line-chart, showing **WinRatio** per season.


In [ ]:
import plotly.graph_objects as go


def plot_driver_vs_teammates(driver_id):
    driver_win_ratios = get_driver_win_ratios(driver_id)
    teammates_win_ratios = get_teammates_win_ratios(driver_id)

    driver_trace = go.Scatter(
        name="Driver",
        x=driver_win_ratios["Year"],
        y=driver_win_ratios["WinRatio"],
        text=driver_win_ratios["DriverName"].iloc[0],
        mode="lines+markers",
        marker=dict(size=8),
        line=dict(color="red"),
    )

    teammates_trace = go.Scatter(
        name="Teammate",
        x=teammates_win_ratios["Year"],
        y=teammates_win_ratios["WinRatio"],
        text=teammates_win_ratios["DriverAbbreviation"],
        textposition="bottom left",
        mode="lines+markers+text",
        marker=dict(size=8),
        line=dict(color="blue"),
        hovertext=teammates_win_ratios["DriverName"],
    )

    layout = go.Layout(
        xaxis_title="Year",
        xaxis=dict(dtick=1),
        yaxis_title="Win ratio",
        yaxis=dict(dtick=0.1),
        yaxis_range=[0, 1],
        height=800,
    )

    fig = go.Figure(data=[driver_trace, teammates_trace], layout=layout)
    fig.show()

## Analyzing Drivers

With the data prepared, the calculations in place, and reusable functions for selecting and visualising individual drivers, we can now explore the results!

To use the helper functions, we first need the **driver_id** of each driver we want to analyze. Although the **driver_id** can be guessed for most drivers, the reliable way is to query the database they originate from.


In [ ]:
results = [
    row[0]
    for row in f1db.execute_raw_sql_query(
        "SELECT id FROM driver WHERE name like '%max verstappen%'"
    )
]

plot_driver_vs_teammates(results[0])